# SOEN363 Project

## Drop the DB Indecies **(Only for Debugging Purposes)**

In [ ]:
DROP INDEX actors_index;
DROP INDEX actors_id_index;
DROP INDEX movies_index;
DROP INDEX movies_count_index;
DROP INDEX movies_rating_index;
DROP INDEX genres_index;
DROP INDEX tag_names_index;

## Drop the DB Views **(Only for Debugging Purposes)**

In [ ]:
DROP VIEW high_ratings;
DROP VIEW low_ratings;
DROP VIEW non_existent;
DROP VIEW no_duplicates;
DROP VIEW all_combinations;
DROP VIEW co_actors;

## Drop the DB tables **(Only for Debugging Purposes)**

In [ ]:
DROP TABLE actors;
DROP TABLE genres;
DROP TABLE tags;
DROP TABLE tag_names;
DROP TABLE movies;

# Assignment Questions
## 2a

In [ ]:
CREATE TABLE movies (
    mid             INTEGER UNIQUE, 
    title           VARCHAR, 
    year            INTEGER, 
    rating          REAL CHECK (rating BETWEEN 0.0 AND 5.0), 
    num_ratings     INTEGER,
    PRIMARY KEY     (mid)
);

CREATE TABLE actors (
    mid             INTEGER REFERENCES movies(mid), 
    name            VARCHAR, 
    cast_position   INTEGER,
    PRIMARY KEY     (mid, name)
);

CREATE TABLE genres (
    mid             INTEGER REFERENCES movies(mid), 
    genre           VARCHAR,
    PRIMARY KEY     (mid, genre)
);

CREATE TABLE tag_names (
    tid             INTEGER, 
    tag             VARCHAR,
    PRIMARY KEY     (tid)
);

CREATE TABLE tags (
    mid             INTEGER REFERENCES movies(mid), 
    tid             INTEGER REFERENCES tag_names(tid),
    PRIMARY KEY     (mid, tid)
);

COPY movies FROM '/srv/soen363/phase1/movies.dat';

COPY actors FROM '/srv/soen363/phase1/actors.dat';

COPY genres FROM '/srv/soen363/phase1/genres.dat';

COPY tag_names FROM '/srv/soen363/phase1/tag_names.dat';

COPY tags FROM '/srv/soen363/phase1/tags.dat';

## 3a

In [ ]:
-- Get the movie titles starring "Daniel Craig", sorted in an ascending alphabetical order
SELECT DISTINCT M.title AS "Movie Titles"
FROM actors A, movies M
WHERE
    M.mid = A.mid AND
    A.name = 'Daniel Craig'
ORDER BY M.title ASC;

## 3b

In [ ]:
-- Get the cast of the movie titled "The Dark Knight", sorted in ascending alphabetical order
SELECT DISTINCT A.name AS "Cast"
FROM actors A, movies M
WHERE
    M.mid = A.mid AND
    M.title = 'The Dark Knight'
ORDER BY A.name ASC;

## 3c

In [ ]:
-- Get the distinct genres in the database, alongside their number of occurrences (only for 1000+ occurrences), sorted ascending numerically in occurrences
SELECT DISTINCT G.genre, COUNT(M.*) AS number_of_movies_in_genre
FROM genres G, movies M
WHERE G.mid = M.mid
GROUP BY G.genre
HAVING COUNT(M.*) > 1000
ORDER BY COUNT(M.*) ASC;

## 3d

In [ ]:
-- For each year, print the movie title, year, and rating, sorted in the ascending order of year and the descending order of movie rating
SELECT M.title, M.year, M.rating
FROM movies M
WHERE M.num_ratings > 0
ORDER BY M.year ASC, M.rating DESC;

## 3e

In [ ]:
-- Get the titles of movies whose tags include at least one occurrence of the word "bad" and another of "good"
SELECT M.title
FROM movies M, tags T, tag_names TN
WHERE 
    M.mid = T.mid AND T.tid = TN.tid AND
    TN.tag IN (
        SELECT TN.tag 
        FROM tag_names TN
        WHERE TN.tag LIKE 'good%'
        UNION
        SELECT TN.tag 
        FROM tag_names TN
        WHERE TN.tag LIKE 'bad%'
    );

## 3f

In [ ]:
-- i. Get the information for the movies with the highest number of ratings, sorted ascending by movie ID
SELECT M.mid, M.title, M.year, M.rating, M.num_ratings
FROM movies M
WHERE M.num_ratings IN (
    SELECT MAX(M.num_ratings)
    FROM movies M
)
ORDER BY M.mid;

In [ ]:
-- ii. Get the information for the movies with the highest rating, sorted ascending by movie ID
SELECT M.mid, M.title, M.year, M.rating, M.num_ratings
FROM movies M
WHERE M.rating IN (
    SELECT MAX(M.rating)
    FROM movies M
)
ORDER BY M.mid;

In [ ]:
-- iii. Get the information for the movies with BOTH the highest rating AND the highest number of ratings (if it exists)

From the data returned from the last two queries, we know that this isn't the case; the movie with the highest rating isn't among the movies with the highest number of ratings.

Highest number of ratings: mid 4201/53125, title Pirates of the Caribbean: At World's End

Highest rating: mid 4311, title 1732 Høtten

In [ ]:
-- iv. Get the information for the movies with the lowest rating, sorted by ascending movie ID
SELECT M.mid, M.title, M.year, M.rating, M.num_ratings
FROM movies M
WHERE M.rating IN (
    SELECT MIN(M.rating)
    FROM movies M
    WHERE M.num_ratings > 0
)
ORDER BY M.mid;

In [ ]:
-- v. Get the information for the movies with BOTH the lowest AND the highest number of ratings (if it exists)

From the data returned from the last query, we know that the lowest rating (for a movie which has been rated more than 0 times) is 1.5
This means that it is not the case for the movie with the highest number of ratings (currently 1'768'593 ratings at 3.8) to be the movie with the lowest rating (currently 3 ratings at 1.5).

In [ ]:
-- vi. Present your findings on the hypothesis: are popular movies (the most ratings) among the best/worst movies (lowest or highest rating score)?

According to the results from the previous queries, it can be safely said that the hypothesis is false for this database.

## 3g

In [ ]:
-- Get the year, title and rating of the lowest-rated and highest-rated movies for each year, between 2005 and 2011 inclusively. If tied, use ascending title

SELECT M2.year, M2.title, M2.rating
FROM movies M2
INNER JOIN (
    SELECT M1.year, MAX(M1.rating) as highest_rating
    FROM movies M1
    WHERE M1.year BETWEEN 2005 AND 2011 AND M1.num_ratings > 0
    GROUP BY M1.year
    ORDER BY M1.year ASC
) highest ON (highest.year = M2.year AND highest.highest_rating = M2.rating)

UNION

SELECT M4.year, M4.title, M4.rating
FROM movies M4
INNER JOIN (
    SELECT M3.year, MIN(M3.rating) as lowest_rating
    FROM movies M3
    WHERE M3.year BETWEEN 2005 AND 2011 AND M3.num_ratings > 0
    GROUP BY M3.year
    ORDER BY M3.year ASC
) lowest ON (lowest.year = M4.year AND lowest.lowest_rating = M4.rating)

ORDER BY year ASC, rating ASC, title ASC

Only movies with more than 0 ratings were considered in this query. As no ratings were given to movies in 2010 and 2011, no movies from those years appear in the results. Removing the constraint will make the results appear, albeit with their ratings being 0.0.

## 3h

In [ ]:
-- High Ratings view to get actors that have participated in a movie with a rating >= 4
CREATE OR REPLACE VIEW high_ratings AS
    SELECT DISTINCT A.name 
    FROM actors A, movies M
    WHERE A.mid = m.mid AND m.rating >= 4; 

-- Low Ratings view to get actors that have participated in a movie with a rating < 4
CREATE OR REPLACE VIEW low_ratings AS
    SELECT DISTINCT A.name 
    FROM actors A, movies M
    WHERE A.mid = m.mid AND m.rating < 4;

-- Get the actors count coming from High Ratings 
SELECT COUNT(*) as result_count
FROM high_ratings;

-- Get the actors count coming from Low Ratings
SELECT COUNT(*) as result_count
FROM low_ratings;

In [ ]:
-- Get the number of No Flop actors in the database
SELECT COUNT(*)
FROM high_ratings
WHERE name NOT IN (
    SELECT name
    FROM low_ratings
);

In [ ]:
-- Get the name and movie count of the Top 10 No Flop actors
SELECT R.name, COUNT(A.*) as movie_count
FROM high_ratings R, actors A
WHERE 
    R.name NOT IN (
        SELECT name
        FROM low_ratings
    ) AND 
    A.name = R.NAME
GROUP BY R.name
ORDER BY COUNT(A.*) DESC
LIMIT 10;

## 3i

In [ ]:
-- Find the actor with the longest longevity
SELECT 
    A.name, 
    MIN(M.year) as start, 
    MAX(M.year) as end, 
    MAX(M.year) - MIN(M.year) as longevity
FROM actors A, movies M
WHERE A.mid = M.mid
GROUP BY A.name 
ORDER BY MAX(M.year) - MIN(M.year) DESC
LIMIT 1;

## 3j

In [ ]:
-- Returns the distinct names of actors who played in at least one movie with Annette Nicole.
CREATE OR REPLACE VIEW co_actors AS
SELECT DISTINCT name
FROM actors 
WHERE mid IN (
    SELECT mid 
    FROM actors
    WHERE name = 'Annette Nicole'
);

-- Print the number of rows in this view. 
SELECT COUNT(name)
FROM co_actors;

In [ ]:
-- Returns all possible combinations of co_actors and the movie ids in which Annette Nicole played.
CREATE OR REPLACE VIEW all_combinations AS
SELECT DISTINCT co_actors.name, actors.mid
FROM co_actors, actors
WHERE actors.mid IN (
    SELECT mid 
    FROM actors
    WHERE name = 'Annette Nicole'
)
ORDER BY co_actors.name;

-- Print the number of rows in this view. 
SELECT COUNT(name)
FROM all_combinations;

In [ ]:
-- Remove all legitimate pairs from all_combinations.
CREATE OR REPLACE VIEW non_existent AS
SELECT * FROM all_combinations
WHERE NOT EXISTS (
    SELECT * 
    FROM actors
    WHERE 
        all_combinations.name = actors.name AND 
        all_combinations.mid = actors.mid
);

-- Print the number of rows in this view.
SELECT COUNT(name)
FROM non_existent;

In [ ]:
-- Eliminate the distinct actors from co_actors that appear in the view non_existent.
-- Print the names of all co_actors except Annette Nicole.

SELECT * FROM co_actors
WHERE NOT EXISTS (
    SELECT * 
    FROM non_existent
    WHERE non_existent.name = co_actors.name
) AND 
NOT co_actors.name = 'Annette Nicole';

## 3k

## 3l

In [ ]:
CREATE OR REPLACE VIEW tags_in_common AS
    WITH tags_count AS (
        SELECT COUNT(tid) 
        FROM tags
    )
    SELECT M2.mid, T2.title, T2.count::REAL / C.count::REAL AS similarity
    FROM tags_count C, movies M2 RIGHT JOIN (
        SELECT M1.mid, M1.title, COUNT(T1.tid)
        FROM tags T1, movies M1
        WHERE 
            T1.tid IN (
                SELECT T.tid
                FROM tags T, movies M
                WHERE 
                    M.title = 'Mr. & Mrs. Smith' AND
                    T.mid = M.mid
            ) AND 
            M1.title != 'Mr. & Mrs. Smith' AND
            T1.mid = M1.mid
        GROUP BY M1.mid, M1.title
        ORDER BY COUNT(T1.tid) DESC
    ) T2
    ON T2.mid = M2.mid
    ORDER BY similarity DESC;

SELECT * FROM tags_in_common;

In [ ]:
CREATE OR REPLACE VIEW actors_in_common AS
    WITH actors_count AS (
        SELECT COUNT(distinct name) 
        FROM actors
    )
    SELECT M2.mid, A2.title, A2.count::REAL / C.count::REAL AS similarity
    FROM actors_count C, movies M2 RIGHT JOIN (
        SELECT M1.mid, M1.title, COUNT(A1.name)
        FROM actors A1, movies M1
        WHERE 
            A1.name IN (
                SELECT A.name
                FROM actors A, movies M
                WHERE 
                    M.title = 'Mr. & Mrs. Smith' AND
                    A.mid = M.mid
            ) AND 
            M1.title != 'Mr. & Mrs. Smith' AND
            A1.mid = M1.mid
        GROUP BY M1.mid, M1.title
        ORDER BY COUNT(A1.name) DESC
    ) A2
    ON A2.mid = M2.mid
    ORDER BY similarity DESC;

SELECT * FROM actors_in_common;

In [ ]:
CREATE OR REPLACE VIEW genres_in_common AS
    WITH genres_count AS (
        SELECT COUNT(distinct genre) 
        FROM genres
    )
    SELECT M2.mid, G2.title, G2.count::REAL / C.count::REAL AS similarity
    FROM genres_count C, movies M2 RIGHT JOIN(
        SELECT M1.mid, M1.title, COUNT(G1.genre)
        FROM genres G1, movies M1
        WHERE 
            G1.genre IN (
                SELECT G.genre
                FROM genres G, movies M
                WHERE 
                    M.title = 'Mr. & Mrs. Smith' AND
                    G.mid = M.mid
            ) AND 
            M1.title != 'Mr. & Mrs. Smith' AND
            G1.mid = M1.mid
        GROUP BY M1.mid, M1.title
        ORDER BY COUNT(G1.genre) DESC
    ) G2
    ON G2.mid = M2.mid
    ORDER BY similarity DESC;

SELECT * FROM genres_in_common;

In [92]:
CREATE OR REPLACE FUNCTION norm(REAL) RETURNS REAL
    AS 
        $$
        DECLARE normal REAL;
        BEGIN
            SELECT (($1 - 2005) / (MAX(M.year) - MIN(M.year))) INTO normal
            FROM movies M;

            RETURN normal;
        END;
        $$
    LANGUAGE plpgsql;

CREATE OR REPLACE VIEW age_gap AS
    SELECT M1.mid, M1.title, norm(M1.year::REAL) as gap
    FROM movies M1
    GROUP BY M1.mid, M1.title
    ORDER BY gap ASC;

SELECT * FROM age_gap;

Commands completed successfully

Commands completed successfully

(10197 row(s) affected)

Displaying Top 5000 rows.

Total execution time: 00:00:07.314

mid,title,gap
6990,The Great Train Robbery,-0.9444444
7065,The Birth of a Nation,-0.8333333
7243,Intolerance: Love's Struggle Throughout the Ages,-0.8240741
48374,Otets Sergiy,-0.8148148
8511,The Immigrant,-0.8148148
3309,A Dog's Life,-0.8055556
6988,Broken Blossoms or The Yellow Man and the Girl,-0.7962963
2821,Male and Female,-0.7962963
2823,"Die Spinnen, 1. Teil - Der Goldene See",-0.7962963
3231,The Saphead,-0.787037


## 3m

In [ ]:
-- Detecting duplicates
SELECT title, year, rating, num_ratings, COUNT(*) occurrences
FROM movies
GROUP BY title, year, rating, num_ratings
HAVING COUNT(*) > 1;

In [ ]:
-- Creating a view that contains no duplicates
CREATE OR REPLACE VIEW no_duplicates AS 
SELECT DISTINCT title, year, rating, num_ratings
FROM movies
GROUP BY title, year, rating, num_ratings;

## 4a

In [ ]:
CREATE INDEX actors_index ON actors(name);
CREATE INDEX actors_id_index ON actors(mid);
CREATE INDEX movies_index ON movies(title);
CREATE INDEX movies_count_index ON movies(num_ratings);
CREATE INDEX movies_rating_index ON movies(rating);
CREATE INDEX genres_index ON genres(genre);
CREATE INDEX tag_names_index ON tag_names(tag);

## 4b

In [ ]:
-- Profiling Results:

-- BEFORE
-- 3A: 0.021s
-- 3B: 0.005s
-- 3C: 0.014s
-- 3D: 0.085s
-- 3E: 0.014s
-- 3F: 0.009s | 0.008s | 0.008s
-- 3G: 0.009s
-- 3H: 0.101s | 0.101s | 0.171s
-- 3I: 0.127s
-- 3J: 0.021s | 0.051s | 0.100s | 0.063s
-- 3K: ?s
-- 3L: ?s
-- 3M: 0.019s

-- AFTER
-- 3A: 0.006s
-- 3B: 0.007s
-- 3C: 0.014s
-- 3D: 0.082s
-- 3E: 0.015s
-- 3F: 0.006s | 0.008s | 0.008s
-- 3G: 0.009s
-- 3H: 0.100s | 0.105s | 0.154s
-- 3I: 0.124s
-- 3J: 0.008s | 0.050s | 0.102s | 0.138s
-- 3K: ?s
-- 3L: ?s
-- 3M: 0.024s

## 4c

## 4d